# Student Behavior Analysis & Face Recognition Training

This notebook trains two models:
1. **Behavior Classification** (Swin Transformer)
2. **Face Recognition** (ArcFace + ResNet50)

## Hardware: Kaggle GPU (P100) - Optimized Settings


In [ ]:
import timm
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader


## 📋 Setup Instructions

**Before running this notebook:**
1. Upload your dataset as a Kaggle Dataset or zip file
2. Enable GPU: Settings → Accelerator → GPU P100
3. Enable Internet: Settings → Internet → ON (for downloading pretrained models)

**Dataset structure expected:**
```
Behaviors_Features/
├── Looking_Forward/
│   ├── ID1/
│   ├── ID2/
│   ├── ID3/
│   └── ID4/
├── Raising_Hand/
├── Reading/
├── Sleeping/
├── Standing/
├── Turning_Around/
└── Writting/
```


In [ ]:
# Install packages
!pip install timm seaborn pillow


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import timm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import List, Tuple, Dict, Optional
import random
import json
import time
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import os
import zipfile
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")


## 📦 Dataset Setup

Upload your dataset and adjust the path below:


In [ ]:
# ===== ADJUST THIS PATH =====
# Option 1: If you uploaded as Kaggle Dataset
DATASET_PATH = "/kaggle/input/your-dataset-name/Behaviors_Features"

# Option 2: If you uploaded a zip file, uncomment below:
# !unzip -q /kaggle/input/your-zip-file/Behaviors_Features.zip -d /kaggle/working/
# DATASET_PATH = "/kaggle/working/Behaviors_Features"

# Verify dataset exists
if Path(DATASET_PATH).exists():
    print(f"✅ Dataset found at: {DATASET_PATH}")
    print(f"   Classes: {[d.name for d in Path(DATASET_PATH).iterdir() if d.is_dir()]}")
else:
    print(f"❌ Dataset NOT found at: {DATASET_PATH}")
    print("   Please adjust DATASET_PATH above!")


## 🗂️ Dataset Class


In [ ]:
class BehaviorDataset(Dataset):
    """Student Behavior Classification Dataset"""
    
    CLASSES = [
        'Looking_Forward',
        'Raising_Hand',
        'Reading',
        'Sleeping',
        'Standing',
        'Turning_Around',
        'Writting'
    ]
    
    def __init__(
        self,
        root_dir: str,
        student_ids: List[str] = None,
        transform: Optional[transforms.Compose] = None,
        augment: bool = True,
        max_samples_per_class: Optional[int] = None
    ):
        self.root_dir = Path(root_dir)
        self.student_ids = student_ids or ['ID1', 'ID2', 'ID3', 'ID4']
        self.max_samples_per_class = max_samples_per_class
        
        # Class mapping
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.CLASSES)}
        self.idx_to_class = {idx: cls for cls, idx in self.class_to_idx.items()}
        
        # Transforms
        if transform:
            self.transform = transform
        else:
            self.transform = self._get_default_transforms(augment)
        
        # Load samples
        self.samples = self._load_samples()
        
        print(f"Loaded {len(self.samples):,} samples from {len(self.student_ids)} students")
        self._print_class_distribution()
    
    def _get_default_transforms(self, augment: bool) -> transforms.Compose:
        """Default transforms for Swin Transformer"""
        if augment:
            return transforms.Compose([
                transforms.Resize((256, 256)),
                transforms.RandomCrop(224),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(10),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        else:
            return transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
    
    def _load_samples(self) -> List[Tuple[Path, int]]:
        """Load all image paths and labels"""
        samples = []
        
        for class_name in self.CLASSES:
            class_dir = self.root_dir / class_name
            if not class_dir.exists():
                print(f"⚠️ Warning: {class_dir} not found, skipping")
                continue
            
            class_samples = []
            class_idx = self.class_to_idx[class_name]
            
            for student_id in self.student_ids:
                student_dir = class_dir / student_id
                if not student_dir.exists():
                    continue
                
                png_files = list(student_dir.rglob('*.png'))
                for img_path in png_files:
                    class_samples.append((img_path, class_idx))
            
            # Limit samples if specified
            if self.max_samples_per_class and len(class_samples) > self.max_samples_per_class:
                class_samples = random.sample(class_samples, self.max_samples_per_class)
            
            samples.extend(class_samples)
        
        random.shuffle(samples)
        return samples
    
    def _print_class_distribution(self):
        """Print class statistics"""
        class_counts = {}
        for _, label in self.samples:
            class_name = self.idx_to_class[label]
            class_counts[class_name] = class_counts.get(class_name, 0) + 1
        
        print("\n📊 Class Distribution:")
        print("-" * 60)
        for class_name in self.CLASSES:
            count = class_counts.get(class_name, 0)
            pct = (count / len(self.samples) * 100) if self.samples else 0
            bar = '█' * int(pct / 2)
            print(f"{class_name:<20} {count:>8,} ({pct:>5.2f}%) {bar}")
        print("-" * 60)
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            image = Image.new('RGB', (224, 224), (0, 0, 0))
        
        if self.transform:
            image = self.transform(image)
        
        return image, label
    
    def get_class_weights(self):
        """Calculate class weights for imbalanced data"""
        class_counts = torch.zeros(len(self.CLASSES))
        for _, label in self.samples:
            class_counts[label] += 1
        
        total = len(self.samples)
        weights = total / (len(self.CLASSES) * class_counts)
        weights = weights / weights.sum() * len(self.CLASSES)
        return weights


In [ ]:
class BehaviorClassifier(nn.Module):
    """Swin Transformer for Behavior Classification"""
    
    def __init__(self, num_classes=7, pretrained=True, model_name='swin_tiny_patch4_window7_224'):
        super().__init__()
        
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=num_classes
        )
        
        self.num_classes = num_classes
        self.model_name = model_name
    
    def forward(self, x):
        return self.backbone(x)

print("✅ Model class defined")


## ⚙️ Training Configuration


In [ ]:
# ===== CONFIGURATION =====
CONFIG = {
    # Training
    'batch_size': 64,           # P100 has 16GB VRAM, can handle large batches
    'num_epochs': 25,           # Full training
    'learning_rate': 1e-4,
    'weight_decay': 1e-4,
    
    # Model
    'model_name': 'swin_tiny_patch4_window7_224',
    'pretrained': True,
    'use_class_weights': True,
    
    # Data
    'num_workers': 2,           # Kaggle works well with 2 workers
    'train_ids': ['ID1', 'ID2', 'ID3'],
    'val_ids': ['ID4'],
    'test_ids': ['ID4'],
    
    # Device
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    
    # Paths
    'save_dir': '/kaggle/working/models'
}

print("📝 Configuration:")
for key, val in CONFIG.items():
    print(f"   {key:<20} {val}")


## 📚 Load Datasets


In [ ]:
print("🔄 Creating datasets...")

train_dataset = BehaviorDataset(
    root_dir=DATASET_PATH,
    student_ids=CONFIG['train_ids'],
    augment=True
)

val_dataset = BehaviorDataset(
    root_dir=DATASET_PATH,
    student_ids=CONFIG['val_ids'],
    augment=False
)

test_dataset = BehaviorDataset(
    root_dir=DATASET_PATH,
    student_ids=CONFIG['test_ids'],
    augment=False
)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

print(f"\n✅ Dataloaders created:")
print(f"   Train batches: {len(train_loader)}")
print(f"   Val batches:   {len(val_loader)}")
print(f"   Test batches:  {len(test_loader)}")


In [ ]:
print("🔄 Creating Swin Transformer model...")

model = BehaviorClassifier(
    num_classes=7,
    pretrained=CONFIG['pretrained'],
    model_name=CONFIG['model_name']
)
model = model.to(CONFIG['device'])

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✅ Model created:")
print(f"   Architecture: {CONFIG['model_name']}")
print(f"   Total params: {total_params:,}")
print(f"   Trainable:    {trainable_params:,}")

# Loss function with class weights
if CONFIG['use_class_weights']:
    class_weights = train_dataset.get_class_weights().to(CONFIG['device'])
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    print(f"\n🎯 Using weighted loss:")
    for i, weight in enumerate(class_weights):
        print(f"   {train_dataset.idx_to_class[i]:<20} {weight:.4f}")
else:
    criterion = nn.CrossEntropyLoss()
    print("\n🎯 Using standard CrossEntropyLoss")

# Optimizer & Scheduler
optimizer = optim.AdamW(
    model.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay']
)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=3,
    verbose=True
)

print(f"\n✅ Optimizer: AdamW (lr={CONFIG['learning_rate']}, wd={CONFIG['weight_decay']})")
print(f"✅ Scheduler: ReduceLROnPlateau (patience=3)")


## 🏋️ Training Loop


In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    
    pbar = tqdm(loader, desc="Training")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * images.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total += images.size(0)
        
        # Update progress bar
        acc = running_corrects.double() / total
        pbar.set_postfix({'loss': f'{loss.item():.4f}', 'acc': f'{acc:.4f}'})
    
    epoch_loss = running_loss / total
    epoch_acc = running_corrects.double() / total
    return epoch_loss, epoch_acc.item()


@torch.no_grad()
def validate(model, loader, criterion, device):
    """Validate model"""
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    
    for images, labels in tqdm(loader, desc="Validation"):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * images.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total += images.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc = running_corrects.double() / total
    return epoch_loss, epoch_acc.item()

print("✅ Training functions defined")


In [ ]:
# Create save directory
Path(CONFIG['save_dir']).mkdir(parents=True, exist_ok=True)

# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'lr': []
}

best_val_acc = 0.0
best_model_path = None

print("\n" + "="*80)
print("🚀 STARTING TRAINING")
print("="*80)
print(f"Device: {CONFIG['device']}")
print(f"Epochs: {CONFIG['num_epochs']}")
print(f"Batch size: {CONFIG['batch_size']}")
print("="*80 + "\n")

start_time = time.time()

for epoch in range(CONFIG['num_epochs']):
    epoch_start = time.time()
    
    print(f"\n{'='*80}")
    print(f"Epoch {epoch + 1}/{CONFIG['num_epochs']}")
    print(f"{'='*80}")
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, CONFIG['device'])
    
    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion, CONFIG['device'])
    
    # Update scheduler
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['lr'].append(current_lr)
    
    epoch_time = time.time() - epoch_start
    
    print(f"\n📊 Results:")
    print(f"   Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"   Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")
    print(f"   LR: {current_lr:.6f} | Time: {epoch_time:.1f}s")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_path = Path(CONFIG['save_dir']) / 'best_behavior_model.pth'
        
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
            'history': history,
            'config': CONFIG
        }, best_model_path)
        
        print(f"   ✅ Saved new best model (acc: {val_acc:.4f})")
    
    # Save checkpoint every 5 epochs
    if (epoch + 1) % 5 == 0:
        checkpoint_path = Path(CONFIG['save_dir']) / f'checkpoint_epoch{epoch + 1}.pth'
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
            'history': history,
            'config': CONFIG
        }, checkpoint_path)
        print(f"   💾 Checkpoint saved: epoch {epoch + 1}")

total_time = time.time() - start_time

print("\n" + "="*80)
print("🎉 TRAINING COMPLETE!")
print("="*80)
print(f"Total time: {total_time / 60:.1f} minutes")
print(f"Best validation accuracy: {best_val_acc:.4f}")
print(f"Best model: {best_model_path}")
print("="*80)

# Save history
history_path = Path(CONFIG['save_dir']) / 'training_history.json'
with open(history_path, 'w') as f:
    json.dump(history, f, indent=2)
print(f"\n💾 Training history saved to: {history_path}")


## 📊 Plot Training Curves


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

epochs_range = range(1, len(history['train_loss']) + 1)

# Loss plot
axes[0].plot(epochs_range, history['train_loss'], 'bo-', label='Train', linewidth=2)
axes[0].plot(epochs_range, history['val_loss'], 'ro-', label='Validation', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training & Validation Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(epochs_range, history['train_acc'], 'bo-', label='Train', linewidth=2)
axes[1].plot(epochs_range, history['val_acc'], 'ro-', label='Validation', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Training & Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

# Learning rate plot
axes[2].plot(epochs_range, history['lr'], 'go-', linewidth=2)
axes[2].set_xlabel('Epoch', fontsize=12)
axes[2].set_ylabel('Learning Rate', fontsize=12)
axes[2].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
axes[2].set_yscale('log')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(Path(CONFIG['save_dir']) / 'training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Training curves saved")


## 🧪 Test Set Evaluation


In [ ]:
print("🔄 Evaluating on test set...")

# Load best model
checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Collect predictions
all_preds = []
all_labels = []
all_probs = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images = images.to(CONFIG['device'])
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())
        all_probs.extend(probs.cpu().numpy())

# Classification report
report = classification_report(
    all_labels,
    all_preds,
    target_names=BehaviorDataset.CLASSES,
    output_dict=True
)

print("\n" + "="*80)
print("📈 TEST SET RESULTS")
print("="*80)
print(f"\n{'Class':<20} {'Precision':>10} {'Recall':>10} {'F1-Score':>10} {'Support':>10}")
print("-"*80)
for class_name in BehaviorDataset.CLASSES:
    metrics = report[class_name]
    print(f"{class_name:<20} {metrics['precision']:>10.4f} {metrics['recall']:>10.4f} "
          f"{metrics['f1-score']:>10.4f} {metrics['support']:>10.0f}")

print("-"*80)
print(f"{'OVERALL ACCURACY':<20} {report['accuracy']:>10.4f}")
print(f"{'Macro Avg F1':<20} {report['macro avg']['f1-score']:>10.4f}")
print(f"{'Weighted Avg F1':<20} {report['weighted avg']['f1-score']:>10.4f}")
print("="*80)


## 🎯 Confusion Matrix


In [ ]:
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=BehaviorDataset.CLASSES,
    yticklabels=BehaviorDataset.CLASSES,
    cbar_kws={'label': 'Count'},
    square=True
)
plt.xlabel('Predicted Label', fontsize=13, fontweight='bold')
plt.ylabel('True Label', fontsize=13, fontweight='bold')
plt.title('Confusion Matrix - Test Set', fontsize=15, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(Path(CONFIG['save_dir']) / 'confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Confusion matrix saved")


## 💾 Save Test Results


In [ ]:
test_results = {
    'report': report,
    'confusion_matrix': cm.tolist(),
    'config': CONFIG,
    'best_val_acc': best_val_acc
}

results_path = Path(CONFIG['save_dir']) / 'test_results.json'
with open(results_path, 'w') as f:
    json.dump(test_results, f, indent=2)

print(f"✅ Test results saved to: {results_path}")


## 📥 Download Trained Model

**Your trained model is saved in `/kaggle/working/models/`**

Files to download:
- `best_behavior_model.pth` - Best model checkpoint
- `training_history.json` - Training metrics
- `test_results.json` - Test set evaluation
- `training_curves.png` - Training visualizations
- `confusion_matrix.png` - Confusion matrix

To download:
1. Click the "Output" tab on the right sidebar
2. Click download on the files you want


In [ ]:
# List all saved files
print("\n📁 Files in output directory:")
print("-" * 60)
for file_path in sorted(Path(CONFIG['save_dir']).glob('*')):
    size_mb = file_path.stat().st_size / (1024 * 1024)
    print(f"   {file_path.name:<40} {size_mb:>8.2f} MB")
print("-" * 60)

print("\n✅ ALL DONE! Ready to download and use in your local system.")
